# Practica 3 - Algorithm based on EP

$ min \, f(\vec{x} ) = -20 \cdot e^{\left(-0.2\sqrt{\frac{1}{n} \sum^{n}_{i=1} x^2_1} \right)} -e^{\left( \frac{1}{n} \sum^n_{i=1} cos(2\pi x_i \right)} + 20 + e$     
donde $|x_i| <= 30$

In [59]:
from math import e,cos,pi
import numpy as np
from numpy.random import normal, uniform

In [166]:
def f(x):
    n = len(x)
    return -20*e**(-0.2*((1/n)*sum(x_i**2 for x_i in x))) - e**((1/n)*sum(cos(2*pi*x_i) for x_i in x)) + 20 + e     

In [45]:
round(f([0,0,0]),15)

0.0

## Algorithm based on EP

In [143]:
def read_input(URL):
    with open(URL) as file_in:
        n = int(file_in.readline())
        pop_size, gens = file_in.readline().split(' ')
        pop_size, gens = int(pop_size), int(gens)
        alpha, epsilon = file_in.readline().split(' ')
        alpha, epsilon = float(alpha), float(epsilon)
        return n, pop_size, gens, alpha, epsilon

read_input("example_input.txt")

(2, 100, 200, 2.0, 0.0001)

solution: [np.array(x_i for x_i in x), float(sigma), f(x)]

In [116]:
def random_solution(n):
    parent = []
    x_s = []
    for i in range(n):
        x_s.append(uniform(-30,30))
    sigma = uniform()
    return [np.array(x_s), sigma, f(x_s)]

In [117]:
random_solution(3)

[array([-19.73806343,  -6.37154658,   2.50674315]),
 0.7708482368748536,
 22.16314655366453]

In [137]:
def selection(new_size, population):
    population.sort(key=lambda x: x[2])
    return population[:new_size]

In [133]:
pop = []
for i in range(5):
    pop.append(random_solution(3))
pop

[[array([ -1.43223793, -19.91708562, -21.99556251]),
  0.0173452755968595,
  21.342885646807304],
 [array([-11.7139206 ,  23.19448837,  23.00510291]),
  0.5414982454610403,
  21.267411954738154],
 [array([-23.48400989, -28.68775207,  24.82521433]),
  0.6521446894221067,
  21.982628040736127],
 [array([15.63786499, 18.7392637 , 20.50116993]),
  0.5155554775118554,
  22.153713675031316],
 [array([ 18.52808572, -12.60096183, -21.13970494]),
  0.9741070584597137,
  22.0369388981194]]

In [138]:
selection(2,pop)

[[array([-11.7139206 ,  23.19448837,  23.00510291]),
  0.5414982454610403,
  21.267411954738154],
 [array([ -1.43223793, -19.91708562, -21.99556251]),
  0.0173452755968595,
  21.342885646807304]]

In [101]:
a = [3,3,41,6,4]

In [103]:
a.sort()
a

[3, 3, 4, 6, 41]

In [150]:
def check_domain(x,lower_limit, upper_limit):
    x = list(x)
    for i in range(len(x)):
        if x[i] < lower_limit:
            x[i] = lower_limit
        elif x[i] > upper_limit:
            x[i] = upper_limit
    return np.array(x)

In [151]:
def EP(n, pop_size, gens, alpha, epsilon):
    parents = [random_solution(n) for i in range(pop_size)]
    for gen in range(gens):
        offspring = []
        for p in parents:
            x, sigma = p[0], p[1]
            sigma = max(epsilon, sigma*(1+alpha*normal(0,1)))
            x = x + sigma * normal(0,1)
            # checar dominio de x_i:
            x = check_domain(x,-30,30)
            
            offspring_i = [x, sigma, f(x)]
            
            offspring.append(offspring_i)
        parents = selection(100, parents+offspring)
    return parents[0]

In [181]:
x, sigma, fx = EP(*read_input("example_input.txt"))
print(list(x))
print('f(x) =',fx)

[-1.1029903960800813, 1.1029904109109405]
f(x) = 4.817113421000567


## Algorithm statistical analysis

In [204]:
from statistics import mean, pstdev

In [191]:
def m_EP(m, n, pop_size, gens, alpha, epsilon):
    m_solutions = []
    for i in range(m):
        EP_sol = EP(n, pop_size, gens, alpha, epsilon)
        m_solutions.append(EP_sol)
    selection(m, m_solutions)
    return m_solutions

In the function above I've already ordered the solutions from best to worst.

In [189]:
m_EP(5,*read_input("example_input.txt"))

[[array([ 0.07751612, -0.07751611]), 0.0001, 0.32242167252493603],
 [array([-0.71974838,  0.71974843]), 0.0001, 3.8588596064684393],
 [array([ 1.15158931, -1.15158928]), 0.0001, 5.592253532245703],
 [array([-1.1786895 ,  1.17868949]),
  0.00020711477094373582,
  6.028043689786383],
 [array([-1.92978026,  0.97874909]), 0.0001, 7.616029650063965]]

In [190]:
_20sols = m_EP(20,*read_input("example_input.txt"))

In [193]:
def median_sol(m,solutions,one_solution=True):
    if m%2 == 0:
        if one_solution:
            return solutions[m//2]
        return solutions[(m//2)-1] + solutions[(m//2)]
    else:
        return solutions[m//2]

In [194]:
median_sol(20,_20sols)

[array([-0.82782009,  0.82782007]), 0.0001, 3.6803508801887372]

In [201]:
def mean_sol_f(solutions):
    return mean([s[2] for s in solutions])

In [203]:
mean_sol_f(_20sols)

4.349865309251344

In [205]:
def std_sol_f(solutions):
    return pstdev([s[2] for s in solutions])

In [206]:
std_sol_f(_20sols)

4.09143363653346

In [207]:
def EP_statistical_analysis(m, n, pop_size, gens, alpha, epsilon):
    m_sols = m_EP(m, n, pop_size, gens, alpha, epsilon)
    print('Best solution:',m_sols[0])
    print('Worst solution:', m_sols[-1])
    print('Median solution:', median_sol(m,m_sols))
    print('Mean solution:', mean_sol_f(m_sols))
    print('Standard deviation:', std_sol_f(m_sols))

In [208]:
m = 30
EP_statistical_analysis(m,*read_input("example_input.txt"))

Best solution: [array([2.10754137e-08, 2.10754137e-08]), 0.0001, 2.886579864025407e-14]
Worst solution: [array([-2.01622461,  2.99631536]), 0.0001, 14.580198371880524]
Median solution: [array([-0.78740115, -0.06524971]), 0.0001950035081890405, 2.151521183701569]
Mean solution: 3.8041200135842446
Standard deviation: 4.05285453659114


### Examples varying only number of decision variables 

#### Example with 5 decision variables

In [210]:
m = 15
EP_statistical_analysis(m, 5, 100, 200, 2.0, 0.0001)

Best solution: [array([-2.28043287e-09, -2.28043287e-09, -2.28043287e-09, -2.28043287e-09,
       -2.28043287e-09]), 0.0001, 4.440892098500626e-16]
Worst solution: [array([ -4.92544826, -19.89782461,   1.00148996, -26.01064171,
       -25.23090968]), 0.00021816936558423123, 20.57547986952672]
Median solution: [array([-1.49833721e-08, -1.49833721e-08, -1.49833721e-08, -1.49833721e-08,
       -1.49833721e-08]), 0.0001, 1.1102230246251565e-14]
Mean solution: 5.657904220428467
Standard deviation: 8.906857537003066


#### Example with 10 decision variables

In [212]:
m = 15
EP_statistical_analysis(m, 10, 100, 200, 2.0, 0.0001)

Best solution: [array([3.03529214e-09, 3.03529214e-09, 3.03529214e-09, 3.03529214e-09,
       3.03529214e-09, 3.03529214e-09, 3.03529214e-09, 3.03529214e-09,
       3.03529214e-09, 3.03529214e-09]), 0.0001, 4.440892098500626e-16]
Worst solution: [array([ -9.81651828,  29.1702713 ,   4.87949639, -24.79442122,
        -6.97924932, -15.99645453,  -0.07453354,   5.8946128 ,
        11.77927372, -26.02693846]), 0.0001, 20.758309070204827]
Median solution: [array([-1.11749503e-08, -1.11749503e-08, -1.11749503e-08, -1.11749503e-08,
       -1.11749503e-08, -1.11749503e-08, -1.11749503e-08, -1.11749503e-08,
       -1.11749503e-08, -1.11749503e-08]), 0.0001, 7.549516567451064e-15]
Mean solution: 1.8402699099841913
Standard deviation: 5.186781728752503


#### Example with 20 dicision variables

In [213]:
m = 15
EP_statistical_analysis(m, 20, 100, 200, 2.0, 0.0001)

Best solution: [array([-6.42801354e-09, -6.42801354e-09, -6.42801354e-09, -6.42801354e-09,
       -6.42801354e-09, -6.42801354e-09, -6.42801354e-09, -6.42801354e-09,
       -6.42801354e-09, -6.42801354e-09, -6.42801354e-09, -6.42801354e-09,
       -6.42801354e-09, -6.42801354e-09, -6.42801354e-09, -6.42801354e-09,
       -6.42801354e-09, -6.42801354e-09, -6.42801354e-09, -6.42801354e-09]), 0.0001, 4.440892098500626e-16]
Worst solution: [array([-0.9345891, -0.9345891, -0.9345891, -0.9345891, -0.9345891,
       -0.9345891, -0.9345891, -0.9345891, -0.9345891, -0.9345891,
       -0.9345891, -0.9345891, -0.9345891, -0.9345891, -0.9345891,
       -0.9345891, -0.9345891, -0.9345891, -0.9345891, -0.9345891]), 0.0001, 3.4228697897789186]
Median solution: [array([1.52268548e-08, 1.52268548e-08, 1.52268548e-08, 1.52268548e-08,
       1.52268548e-08, 1.52268548e-08, 1.52268548e-08, 1.52268548e-08,
       1.52268548e-08, 1.52268548e-08, 1.52268548e-08, 1.52268548e-08,
       1.52268548e-08, 1.52268